# Topics

- Graph
- connectivity
- maximum flow

In [28]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [10]:
with open("25-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'mgd: fzt\nnqf: bmz\nvtc: mmq mmt vcb\nbjj: npr stt kgv\njvn: bjk bvn\ndng: vcq pmd\nblf: pxz\ntxd: gnl qhf kmp qkl czq\nvfm: nnh\nldz: tzz\nfqc: xzj hbn klt hpb\njgc: rsq\nfpt: ctx\ncht: dnk knj hpd qml\nqjq: rhr xxz\nczd: fpc\nzbl: jbr bnx pmk dvs dvv lnb\nsrn: stt\nrgm: lxt cvc\nnhg: zbd mvf jvl\nrtl: jlf\nfkp: nlb rnh mnb\nrtx: mpf sxv dfr dkt\ngmq: vfs gtf xrb\nftj: nzk xbg\ntfs: mrp vth cgl nvn\nmrl: kmm\ntzv: gjf lkc\nxbg: bst\ntkg: bft\nnzr: gzq bxd bgz qhn\nbvn: shn\nqhf: bnc tpr\ndtd: sld hzb dfm ngl\nbrn: qzn jzn\ngzs: hxm\nfnr: slk sjf blh lxq\nfbj: mqj xbj jcp fmk\nmrd: knd\nbqs: nrz hdr xjz gtz\nftq: tzr cmf mzn\ntdc: clc nlb fjj\nfcz: vdg\ngsv: hcz\nzcm: qdr nmf bvk\nqnv: lnh jhf hjj\nkdh: vxv xgc\nzhh: jvp gxz\nfdv: bbg vxv chd\nsjz: ksm fhp cfk\nhxv: xgc brx xds\nbvr: lkc tdq\nhkb: skd xds jtf\nzqk: txq tzn\nvfd: ggl dbv bxh\nmhx: qfs fzm cfs msh\nmnp: gkd\njjt: xlv lmm gsx jnd kss\nplg: xrc bqx gxt gsn\nchs: dcv fxl kjr tsj\ncqk: hvs\nnbp: jrm pvj\nbgz: cdg\ndqn: skz\

In [8]:
test_data_raw = r"""jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr"""


def preprocess_data (data):
    # dtype='U10'
    def parse_row(row):
        l = row.split(": ")[0]
        k = row.split(": ")[1].split(" ")
        return (l,k)

    rows = [parse_row(row) for row in data.split("\n")]
    return dict(rows)

test_data = preprocess_data(test_data_raw)
display(test_data)

{'jqt': ['rhn', 'xhk', 'nvd'],
 'rsh': ['frs', 'pzl', 'lsr'],
 'xhk': ['hfx'],
 'cmg': ['qnr', 'nvd', 'lhk', 'bvb'],
 'rhn': ['xhk', 'bvb', 'hfx'],
 'bvb': ['xhk', 'hfx'],
 'pzl': ['lsr', 'hfx', 'nvd'],
 'qnr': ['nvd'],
 'ntq': ['jqt', 'hfx', 'bvb', 'xhk'],
 'nvd': ['lhk'],
 'lsr': ['lhk'],
 'rzs': ['qnr', 'cmg', 'lsr', 'rsh'],
 'frs': ['qnr', 'lhk', 'lsr']}

In [11]:
data = preprocess_data(data_raw)
data

{'mgd': ['fzt'],
 'nqf': ['bmz'],
 'vtc': ['mmq', 'mmt', 'vcb'],
 'bjj': ['npr', 'stt', 'kgv'],
 'jvn': ['bjk', 'bvn'],
 'dng': ['vcq', 'pmd'],
 'blf': ['pxz'],
 'txd': ['gnl', 'qhf', 'kmp', 'qkl', 'czq'],
 'vfm': ['nnh'],
 'ldz': ['tzz'],
 'fqc': ['xzj', 'hbn', 'klt', 'hpb'],
 'jgc': ['rsq'],
 'fpt': ['ctx'],
 'cht': ['dnk', 'knj', 'hpd', 'qml'],
 'qjq': ['rhr', 'xxz'],
 'czd': ['fpc'],
 'zbl': ['jbr', 'bnx', 'pmk', 'dvs', 'dvv', 'lnb'],
 'srn': ['stt'],
 'rgm': ['lxt', 'cvc'],
 'nhg': ['zbd', 'mvf', 'jvl'],
 'rtl': ['jlf'],
 'fkp': ['nlb', 'rnh', 'mnb'],
 'rtx': ['mpf', 'sxv', 'dfr', 'dkt'],
 'gmq': ['vfs', 'gtf', 'xrb'],
 'ftj': ['nzk', 'xbg'],
 'tfs': ['mrp', 'vth', 'cgl', 'nvn'],
 'mrl': ['kmm'],
 'tzv': ['gjf', 'lkc'],
 'xbg': ['bst'],
 'tkg': ['bft'],
 'nzr': ['gzq', 'bxd', 'bgz', 'qhn'],
 'bvn': ['shn'],
 'qhf': ['bnc', 'tpr'],
 'dtd': ['sld', 'hzb', 'dfm', 'ngl'],
 'brn': ['qzn', 'jzn'],
 'gzs': ['hxm'],
 'fnr': ['slk', 'sjf', 'blh', 'lxq'],
 'fbj': ['mqj', 'xbj', 'jcp', 'fmk'

In [40]:
def solution (data, verbose=False):

    def max_flow_min_cut(G, s, t):
        pass

    collisions = 0

    nodes = list(set(list(data.keys()) + [node for val in data.values() for node in val]))

    # capacity matrix
    graph = np.zeros((len(nodes), len(nodes)), dtype=int)
    for node, edges in data.items():
        for node_b in edges:
            graph[nodes.index(node), nodes.index(node_b)] = 1
            graph[nodes.index(node_b), nodes.index(node)] = 1

    # for A, B in combinations(range(len(data)), 2):
    # fix one vertex to reduce complexity
    fixed = nodes[0]
    other = nodes[1:]
    comp_A = [fixed]
    comp_B = []
    for A, B in zip([fixed]*len(other), other):
        result = maximum_flow(csr_matrix(graph), nodes.index(A), nodes.index(B))     
        max_flow = result.flow_value
        if max_flow == 3:
            comp_B.append(B)
        else:
            comp_A.append(B)

    print(len(comp_A), comp_A)
    print(len(comp_B), comp_B)

    return len(comp_A) * len(comp_B)



sol = solution(data)
# sol = solution(test_data, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


718 ['ksj', 'dbj', 'pht', 'ngh', 'vsq', 'qdr', 'lkc', 'cxj', 'ftt', 'dmq', 'xgp', 'dpz', 'pdj', 'tzv', 'pqc', 'fzz', 'lbf', 'kjd', 'bmh', 'jsl', 'nfh', 'qjp', 'ctr', 'qlc', 'ddp', 'kms', 'tls', 'vqc', 'dsh', 'rdb', 'nxx', 'jcq', 'cgj', 'nzz', 'svq', 'qhb', 'xgb', 'mcm', 'pgl', 'gcm', 'vmx', 'xrg', 'jkr', 'grt', 'fmt', 'ksm', 'rjq', 'cth', 'xnq', 'jtd', 'hdr', 'dvs', 'jtg', 'szv', 'bhm', 'tqm', 'ppd', 'qvl', 'glp', 'tzn', 'sjz', 'mnz', 'tbv', 'ctf', 'mfv', 'bzs', 'zds', 'vlg', 'tqb', 'kml', 'cqs', 'gjf', 'zmc', 'dzr', 'rmd', 'vcr', 'ckb', 'bcv', 'jdl', 'qzf', 'xks', 'dqp', 'rmv', 'lgb', 'plb', 'fjj', 'bvh', 'jff', 'dlx', 'gtz', 'jbr', 'tll', 'zgd', 'nzc', 'gvj', 'gsn', 'vdg', 'snc', 'nlk', 'tzz', 'bxn', 'qfd', 'gnl', 'bvt', 'mlv', 'qxp', 'cjz', 'fqp', 'rxc', 'ctp', 'bln', 'mfb', 'czd', 'rzv', 'qdt', 'qct', 'trp', 'vmt', 'jjn', 'brn', 'tjb', 'zfr', 'zjr', 'mpr', 'bmj', 'pvl', 'xkk', 'lhp', 'gsc', 'tlx', 'nhp', 'xbb', 'jhz', 'lqs', 'snq', 'fnb', 'thf', 'gvk', 'mnp', 'vbk', 'ngv', 'cbp', '

# Part 2